In [1]:
import pandas as pd
import numpy as np
# from __future__ import print_function
import cplex
from cplex.exceptions import CplexSolverError
from cplex import SparsePair
from cplex.six.moves import zip
import time
import numba
from ortools.algorithms import pywrapknapsack_solver
from numba import jit

In [2]:
def data(ty,problem):
    
    f = open("gap_%s/%s.txt"%(ty,problem), 'r')
    data = f.readlines()
    f.close()

    records = []
    for line in data:
        record = [int(field) for field in line.strip().lstrip('[').rstrip(']').split()]
        records.append(record)

    size = records[0]
    agent = size[0]
    job = size[1]

    c = []
    a = []
    b = []
    for i in range(len(records)-1) :
        if len(c) < job*agent:
            c.extend(records[i+1])
        elif len(c) >= job*agent and len(a)< job*agent :
            a.extend(records[i+1])
        else :
            b.extend(records[i+1])

    c = np.array(c,dtype=int).reshape((agent,job))
    a = np.array(a,dtype=int).reshape((agent,job))
    b = np.array(b)
    
    return a,b,c,agent,job


In [3]:

def General_CG(a,b,c,agent,job):
    
    K = range(1)
    var = list(range(agent))

    M = cplex.Cplex()

    x_i_k = lambda i,k: 'x_%d_%d' % (i,k)
    x = [x_i_k(i,k) for i in range(1) for k in K]

    dummy = float(sum(np.sum(c,axis=1)))

    M.variables.add(
        lb = [0] * len(x),
        ub = [1] * len(x),
        names = x,
        obj =  [dummy],
        types = ['C'] * len(x)
    )


    M.linear_constraints.add(
        lin_expr= [
            cplex.SparsePair(
                ind =[x_i_k(i,k) for i in range(1) for k in K], 
                val = [1.0]
                )
        for j in range(job)
        ],
        senses=["G" for j in range(job)],
        rhs=[1.0 for j in range(job)] ,
        names=['assignment_%d' % (j) for j in range(job)])


    
    M.linear_constraints.add(
        lin_expr= [
            cplex.SparsePair(
                ind =[x_i_k(0,k) for k in K], 
                val = [0] * len(K)
                )
        for i in range(agent)
        ],
        senses=["L" for i in range(agent)],
        rhs=[1.0 for i in range(agent)] )

 
    M.objective.set_sense(M.objective.sense.minimize)    
    M.set_log_stream(None)
    M.set_error_stream(None)
    M.set_warning_stream(None)
    M.set_results_stream(None)

    def subproblem(a,b,c,agent,job,ag):
        S = cplex.Cplex()

        pi = np.min(c,axis=0)


        d_j = lambda j: 'd_%d' % (j)
        d = [d_j(j) for j in range(job)]


        S.variables.add(
            obj = [int(val) for val in list(pi - np.sum(c,axis=0))],
            types = ['B'] * len(d),
            names = d
        )


        S.linear_constraints.add(
            lin_expr = [
                cplex.SparsePair(
                    ind = [d_j(j) for j in range(job)], 
                    val = [int(v) for v in list(a[ag])]
                    )
            ],
            senses = ["L"],
            rhs = [int(b[ag])] )


        S.objective.set_sense(S.objective.sense.maximize)
        S.set_log_stream(None)
        S.set_error_stream(None)
        S.set_warning_stream(None)
        S.set_results_stream(None)
        
        return S,d

    start = time.time()
    mt = 0
    st = 0
    ite = 0

    criterion = True

    while criterion:

        criterion = False

        for ag in range(agent):   
            
            ite+=1
            M.set_problem_type(M.problem_type.LP)
            ct = time.time()
            M.solve()
            mt += time.time() - ct

            pi = list(M.solution.get_dual_values())[:job]
            
            S,d = subproblem(a,b,c,agent,job,ag)

            S.objective.set_linear(list(zip(list(range(len(d))),list(np.array(pi) -np.array(c[ag])))))

            pt = time.time()
            S.solve()
            st += time.time() - pt
            
            print(ag,'S obj val:',S.solution.get_objective_value())
            print('M dual:',-list(M.solution.get_dual_values())[job+ag])

            if S.solution.get_objective_value()-0.000001 > -list(M.solution.get_dual_values())[job+ag]:
                criterion = True
                newsub = S.solution.get_values()
                idx = M.variables.get_num()
                M.variables.add(obj=[np.array(S.solution.get_values()).T @ c[ag]])
                M.linear_constraints.set_coefficients(list(zip(list(range(job)),
                                                                 [idx] * job,
                                                                 newsub)))

                M.linear_constraints.set_coefficients(job+ag, idx, 1.0)
                var.append(idx)

    
    M.set_problem_type(M.problem_type.LP)
    ct = time.time()
    M.solve()
    mt += time.time()- ct
    tt = time.time()- start
    
    return ite,M,S,mt,st,tt


In [4]:
@jit
def sols(w,c):
    i = len(c)-1
    currentW =  len(c[0])-1

    marked = np.zeros(i+1)

    while i >= 0 and currentW >=0:
        if (i==0 and c[i][currentW] >0 )or c[i][currentW] != c[i-1][currentW]:
            marked[i] =1
            currentW = currentW-w[i]
        i = i-1
    return marked


@jit
def Knapsack1(v, w, W):

    n = len(v)

    c = np.zeros((n,W+1))

    for i in range(n):
        for j in range(W+1):
            if (w[i] > j):
                c[i][j] = c[i-1][j]
            else:
                c[i][j] = max(c[i-1][j],v[i] +c[i-1][j-w[i]])

    return c[n-1][W],sols(w,c)

@jit
def Knapsack2(val, wt, W): 

    n = len(val)
    K = np.zeros((n+1,W+1))
    for i in range(n + 1): 
        for w in range(W + 1): 
            if i == 0 or w == 0: 
                K[i][w] = 0
            elif wt[i - 1] <= w: 
                K[i][w] = max(val[i - 1] 
                + K[i - 1][w - wt[i - 1]], 
                        K[i - 1][w]) 
            else: 
                K[i][w] = K[i - 1][w] 
                
    res = K[n][W] 
    sol = np.zeros(n)
    w = W 
    print(W)
    for i in range(n, 0, -1): 
        if res <= 0: 
            break
        elif res == K[i - 1][w]: 
            continue
        else: 
            sol[i-1] = 1
            res = res - val[i - 1] 
            w = w - wt[i - 1] 
    return K[n][W],sol


def Knapsack(val, wt, W): 
    
    wt = [wt]
    W = [W]
    solver = pywrapknapsack_solver.KnapsackSolver(
      pywrapknapsack_solver.KnapsackSolver.
      KNAPSACK_DYNAMIC_PROGRAMMING_SOLVER,
      'test')
    
    solver.Init(val, wt, W)
    computed_value = solver.Solve()
    
    sol = np.zeros(len(val))
    for x in range(len(val)):
        if solver.BestSolutionContains(x):
            sol[x] = 1.

    return computed_value, sol

def DP_General_CG(a,b,c,agent,job):
    
    K = range(1)
    var = list(range(agent))

    M = cplex.Cplex()

    x_i_k = lambda i,k: 'x_%d_%d' % (i,k)
    x = [x_i_k(i,k) for i in range(1) for k in K]

    dummy = float(sum(np.sum(c,axis=1)))

    M.variables.add(
        lb = [0] * len(x),
        ub = [1] * len(x),
        names = x,
        obj =  [dummy],
        types = ['C'] * len(x)
    )



    M.linear_constraints.add(
        lin_expr= [
            cplex.SparsePair(
                ind =[x_i_k(i,k) for i in range(1) for k in K], 
                val = [1.0]
                )
        for j in range(job)
        ],
        senses=["G" for j in range(job)],
        rhs=[1.0 for j in range(job)] ,
        names=['assignment_%d' % (j) for j in range(job)])


    
    M.linear_constraints.add(
        lin_expr= [
            cplex.SparsePair(
                ind =[x_i_k(0,k) for k in K], 
                val = [0] * len(K)
                )
        for i in range(agent)
        ],
        senses=["L" for i in range(agent)],
        rhs=[1.0 for i in range(agent)] )

 
    M.objective.set_sense(M.objective.sense.minimize)    
    M.set_log_stream(None)
    M.set_error_stream(None)
    M.set_warning_stream(None)
    M.set_results_stream(None)

    start = time.time()
    mt = 0
    st = 0
    ite = 0

    criteria = [True]*agent

    while any(criteria):
        for ag in range(agent):   
            
            if criteria[ag] == True:

                ite+=1

                M.set_problem_type(M.problem_type.LP)
                ct = time.time()

                if ag == 5:
                    M.write('m.lp')
                M.solve()
                mt += time.time() - ct

                pi = list(M.solution.get_dual_values())[:job]

                w = list(a[ag])
                v = list(np.array(pi) -np.array(c[ag]))
                W = int(b[ag])

                pt = time.time()

                S_obj,sol = Knapsack(v,w,W)

                st += time.time() - pt

                if S_obj - 0.00001 > -list(M.solution.get_dual_values())[job+ag]:
                    criteria[ag] = True
                    newsub = sol
                    idx = M.variables.get_num()
                    M.variables.add(obj=[float(np.array(sol).T @ c[ag])])
                    M.linear_constraints.set_coefficients(list(zip(list(range(job)),
                                                                     [idx] * job,
                                                                     newsub)))

                    M.linear_constraints.set_coefficients(job+ag, idx, 1.0)
                    var.append(idx)
                else :
                    criteria[ag] = False
    
    M.set_problem_type(M.problem_type.LP)
    ct = time.time()
    M.solve()
    mt += time.time()- ct
    tt = time.time()- start
    
    return ite,M,mt,st,tt


In [5]:

def Stabilization(a,b,c,agent,job):
    
    K = range(1)
    var = list(range(agent))
    eps = 0.1

    M = cplex.Cplex()

    x_i_k = lambda i,k: 'x_%d_%d' % (i,k)
    x = [x_i_k(i,k) for i in range(1) for k in K]

    dummy = float(sum(np.sum(c,axis=1)))

    M.variables.add(
        lb = [0] * len(x),
        ub = [1] * len(x),
        names = x,
        obj =  [dummy],
        types = ['C'] * len(x)
    )


    gp_j = lambda j: 'gp_%d' % (j)

    gp = [gp_j(j) for j in range(job)]

    M.variables.add(
        lb = [0] * len(gp),
        ub = [eps] * len(gp),
        names = gp,
        obj =  [100] * len(gp),
        types = ['C'] * len(gp)
    )


    gm_j = lambda j: 'gm_%d' % (j)

    gm = [gm_j(j) for j in range(job)]

    M.variables.add(
        lb = [0] * len(gm),
        ub = [eps] * len(gm),
        names = gm,
        obj =  [-100] * len(gm),
        types = ['C'] * len(gm)
    )

    yp_i = lambda i: 'yp_%d' % (i)

    yp = [yp_i(i) for i in range(agent)]

    M.variables.add(
        lb = [0] * len(yp),
        ub = [eps] * len(yp),
        names = yp,
        obj =  [100] * len(yp),
        types = ['C'] * len(yp)
    )

    ym_i = lambda i: 'ym_%d' % (i)

    ym = [ym_i(i) for i in range(agent)]

    M.variables.add(
        lb = [0] * len(ym),
        ub = [eps] * len(ym),
        names = ym,
        obj =  [-100] * len(ym),
        types = ['C'] * len(ym)
    )

    M.linear_constraints.add(
        lin_expr= [
            cplex.SparsePair(
                ind = x + [gp_j(j)] + [gm_j(j)] , 
                val = [1.0] + [1.0, -1.0]
                )
        for j in range(job)
        ],
        senses=["G" for j in range(job)],
        rhs=[1.0 for j in range(job)] ,
        names=['assignment_%d' % (j) for j in range(job)])


    M.linear_constraints.add(
        lin_expr= [
            cplex.SparsePair(
                ind = x + [yp_i(i)] + [ym_i(i)] , 
                val = [0] * len(K) + [1.0] + [-1.0]
                )
        for i in range(agent)
        ],
        senses=["L" for i in range(agent)],
        rhs=[1.0 for i in range(agent)] )


    M.objective.set_sense(M.objective.sense.minimize)    
    
    M.set_log_stream(None)
    M.set_error_stream(None)
    M.set_warning_stream(None)
    M.set_results_stream(None)

    start = time.time()
    mt = 0
    st = 0
    ite = 0

    criteria = [True]*agent
    t_total = time.time()

    while any(criteria):

        for ag in range(agent):   
            if criteria[ag] == True:
                
                ite+=1

                M.set_problem_type(M.problem_type.LP)
                M.write('m.lp')
                ct = time.time()
                M.solve()
                mt += time.time() - ct

                pi = list(M.solution.get_dual_values())[:job]
                phi =  list(M.solution.get_dual_values())[job:]

                w = list(a[ag])
                v = list(np.array(pi) -np.array(c[ag]))
                W = int(b[ag])

                pt = time.time()
                S_obj,sol = Knapsack(v,w,W)
                st += time.time() - pt
#                 print('S',S_obj)
#                 print('M',list(M.solution.get_dual_values())[job+ag])

                if (S_obj-0.000001 > -list(M.solution.get_dual_values())[job+ag]) or eps != 0:
                    criteria[ag] = True

                    M.objective.set_linear(zip(gp+gm+yp+ym , pi+list(-np.array(pi))+phi+list(-np.array(phi))))

                    newsub = sol
                    idx = M.variables.get_num()
                    M.variables.add(obj=[sol.T @ c[ag]])
                    M.linear_constraints.set_coefficients(list(zip(list(range(job)),
                                                                     [idx] * job,
                                                                     newsub)))

                    M.linear_constraints.set_coefficients(job+ag, idx, 1.0)
                    var.append(idx)

                    if ite % 100 == 0 :
                        eps *= 0.1
                        if ite == 600 :
                            eps =0
                        
                        for dv in gm+gp+ym+yp:
                            M.variables.set_upper_bounds(dv , eps )
                            
                        

                else :
                    criteria[ag] = False



    M.set_problem_type(M.problem_type.LP)

    M.solve()


    tt = time.time()- t_total
    return ite,M,mt,st,tt



In [6]:

K_results = {}
S_results = {}
problems = ['d05100','d10100','d10200','d20100','d20200','e05100','e10100','e10200','e20100','e20200']
for problem in problems : 
    
    ty = problem[0]
    
    a,b,c,agent,job = data(ty, problem)
    
    ite,M,mt,st,tt = DP_General_CG(a,b,c,agent,job)
    per = mt/(st+mt)
    K_results[problem] = ['Kelly',ite,mt,st,tt,per]
    
    
    ite_s,M_s,mt,st,tt = Stabilization(a,b,c,agent,job)
    per = mt/(st+mt)
    S_results[problem] = ['Stab.',ite_s,mt,st,tt,per]
    
    print(problem, ' done!')

d05100  done!
d10100  done!
d10200  done!
d20100  done!
d20200  done!
e05100  done!
e10100  done!
e10200  done!
e20100  done!
e20200  done!


In [10]:
re = pd.DataFrame(K_results)
re = re.transpose()
name = ['method','iteration','M','S','total','M_per']
re.columns = name

re

,method,iteration,M,S,total,M_per
d05100,Kelly,1275,2.55126,0.751681,3.89494,0.772421
d10100,Kelly,923,1.43239,0.201322,2.05771,0.87677
d10200,Kelly,2712,27.8598,2.93016,32.5287,0.904834
d20100,Kelly,762,0.774554,0.0817223,1.14607,0.904561
d20200,Kelly,2007,11.7037,0.721792,13.6355,0.94191
e05100,Kelly,2402,5.42581,0.466124,6.78356,0.920888
e10100,Kelly,1171,1.68425,0.129978,2.25095,0.928356
e10200,Kelly,3631,47.0664,1.26311,50.6389,0.973865
e20100,Kelly,941,0.939118,0.0819547,1.36652,0.919737
e20200,Kelly,2514,13.5473,0.46271,15.4767,0.966973


In [11]:
re = pd.DataFrame(S_results)
re = re.transpose()

re.columns = name

re

,method,iteration,M,S,total,M_per
d05100,Stab.,1220,4.61856,0.764944,9.78486,0.85791
d10100,Stab.,921,3.1145,0.201951,6.23328,0.939106
d10200,Stab.,2709,42.5438,3.15288,66.4399,0.931004
d20100,Stab.,716,1.99841,0.0820088,4.19832,0.960581
d20200,Stab.,2074,22.7736,0.847104,36.7094,0.964137
e05100,Stab.,2076,6.59324,0.434604,16.1736,0.93816
e10100,Stab.,1151,3.05467,0.138083,6.80993,0.956751
e10200,Stab.,3719,65.5724,1.35226,99.2406,0.979794
e20100,Stab.,888,1.98747,0.0841887,4.73896,0.959362
e20200,Stab.,2592,22.2178,0.520719,39.4815,0.9771
